ASSUMPTIONS/DOCUMENTATION:

    - A null section value indicates that all sections share that exam date and time.
    - Some ARCH courses list section as "80". No clue what this means.
    - The process uses the FinalsBySubject.pdf document from the RPI website.
        - This pdf should have columns Department, Course, Location, Date, and Grades Due (although the first and last don't matter)
        - It should also be titled at the top of each page with Season Year followed by any amount more text (doesn't matter)
        - If the above are not true, small modifications must be made to the process

TODO:

    - make grades due not break the program - maybe need proper exam doc to handle
    - make 9PM end time not get +12hrs

In [61]:
from pypdf import PdfReader
import os
from datetime import datetime
import pandas as pd
import re
import calendar
import logging

In [62]:
logging.basicConfig(filename="log.log",format='%(asctime)s %(message)s',filemode='w')

In [63]:
# Construct a dictionary to get the number of a month from it's word
months = list(calendar.month_name)
months = [x.lower() for x in months]

# turns a time, day of month, month, and year into one datetime object for the table
# this is used to get the start and end times for an exam
def handle_times(input_text,day,month,year):
    # Regex to get the hour and minute as seperate values from a string of the format HH:MM AM or HH:MM PM
    nums = re.findall(r'\d+', input_text)
    nums = [int(x) for x in nums]
    # Instead of trying to track AM/PM we instead use the logic that exams only happen between 8AM - 6PM and convert to military time
    # This is done because RPI likes to have typos such as 8:00 M instead of 8:00 PM making the AM/PM values unreliable
    if nums[0] < 8:
        nums[0] = nums[0] + 12
    try:
        month_num = months.index(month.lower())
    except:
        # March is default in case of error: No exams in March
        month_num = 3
        logging.error('Month Parsing in handle_times() raised an error')
    # Construct and return the datetime object
    text = year + str(month_num) + day + str(nums[0]) + ":"+ str(nums[1])
    format = '%Y%m%d%H:%M'
    time = datetime.strptime(text, format)
    return time

In [64]:
def parser():
    files = [f for f in os.listdir('.') if os.path.isfile(f)]
    
    reader = PdfReader("finals_by_subject.pdf")
    number_of_pages = len(reader.pages)
    
    db_lines = []

    # Process the document page by page
    for page in reader.pages:
        text = page.extract_text(extraction_mode="layout")
        # Process the text to handle the following cases:
        #   -If the location is "TBA" we replace this with "TBA TBA" since location is Room RoomNumber
        #   -If the location is "ONLINE" we replace this with "ONLINE NA" for the same reason (Room=ONLINE,RoomNumber=NA) 
        #   -If there are "/" in the text we remove surrounding spaces so they don't cause issues (multiple different issues)
        #   -Remove the word "SECTIONS" as it's superflous and inconsistent
        text = text.replace(" / ","/").replace("TBA","TBA TBA").replace("ONLINE","ONLINE NA").replace("(ALL ","(ALL").replace("SECTIONS ","")
        # Split text into lines, get the season (Fall,Summer,Spring) and year (20__) from the first line, then remove the first three lines since they are header/bank
        text = text.split('\n')
        for_year = text[0].split(" ")
        for_year = [x for x in for_year if x!='']
        season = for_year[0]
        year = for_year[1]
        text.pop(0)
        text.pop(0)
        text.pop(0)
        # Remove a fourth line for the first page only since it has the column headers
        if "DEPARTMENT" in text[0] and "COURSE" in text[0]:
            text.pop(0)
    
        # Now, parse the lines
        for line in text:
    
            # Remove (in SQL syntax) anything like "(NEEDS%)" because a few random courses say (NEEDS 6 HR BLOCK) or something along those lines
            while "(NEEDS" in line:
                tmp = line[line.index("(NEEDS"):]
                line = line[0:line.index("(NEEDS")] + line[line.index("(NEEDS")+tmp.index(")")+1:]
    
            # Clean up the line and remove department
            line = line.strip()
            line = line.split(" ")
            line = [x for x in line if x != '']
            line.pop(0)
    
            
            first_num = -1
            for i in range(len(line)):
                if any(char.isdigit() for char in line[i]):
                    first_num = i
                    break
            for i in range(first_num-1):
                line.pop(0)
            major = line[0]
            line.pop(0)
            course_string = line[0]
            courses = []
            while "/" in course_string:
                i = course_string.index("/")
                courses.append(course_string[0:i])
                course_string = course_string[i+1:len(course_string)]
            courses.append(course_string)
            line.pop(0)
        
            time2 = line[len(line)-1]
            line.pop(len(line)-1)
            line.pop(len(line)-1)
            time1 = line[len(line)-1]
            line.pop(len(line)-1)
            line.pop(len(line)-1)
            day = line[len(line)-1]
            line.pop(len(line)-1)
            month = line[len(line)-1]
            line.pop(len(line)-1)
            weekday = line[len(line)-1].replace(",",'')
            line.pop(len(line)-1)
            room = line[len(line)-1]
            line.pop(len(line)-1)
            building = line[len(line)-1]
            line.pop(len(line)-1)

        
            # get the sections from the remainder and fix some formatting (take out of parens and remove commas and ampersands)
            sections = [x.replace(",","").replace("(","").replace(")","") for x in line if x!=',' and x!='&']
            # if an entry is info for all sections of a class, write that and skip the rest
            all = False
            for section in sections:
                if "ALL" in section:
                    db_lines.append([season,year,major, course,None,handle_times(time1,day,month,year),handle_times(time2,day,month,year),building,room])
                    all = True
            if all:
                continue
            fixed_sections = []
            # Create seperate section entries for all sections within a range ([01-05] becomes [01,02,03,04,05])
            for section in sections:
                if '-' in section:
                    num1 = int(section[:section.index("-")])
                    num2 = int(section[section.index("-")+1:])
                    sections.remove(section)
                    for i in range(num1,num2+1):
                        fixed_sections.append(i)
                else:
                    fixed_sections.append(int(section))
            sections = fixed_sections
            # Adds all the entries into the array
            for section in sections:
                for course in courses:
                    db_lines.append([season,year,major, course,int(section),handle_times(time1,day,month,year),handle_times(time2,day,month,year),building,room])
    return db_lines

In [65]:
def display_and_write_csv(db_lines):
    try:
        # Place into pandas dateframe (not needed but useful for testing & makes writing to csv easier
        df = pd.DataFrame(columns=('Season','Year','Major','Course','Section','Start','End','Building','Room_Number'))
        for i in range(len(db_lines)):
            df.loc[i] = db_lines[i]
        # standardize datetimes
        df['Start'] = pd.to_datetime(df['Start'])
        df['End'] = pd.to_datetime(df['End'])
        pd.set_option('display.max_rows', 500)
        display(df)
        # write to output csv
        df.to_csv('out.csv')
        logging.info("WROTE OUTPUT TO CSV")
    except:
        logging.error("FAILURE IN display_and_write_csv()")

In [66]:
db_lines = parser()
display_and_write_csv(db_lines)

,Season,Year,Major,Course,Section,Start,End,Building,Room_Number
0,FALL,2024,ARCH,2150,1,2024-12-16 11:30:00,2024-12-16 14:30:00,ONLINE,NA
1,FALL,2024,ARCH,2160,1,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
2,FALL,2024,ARCH,2160,2,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
3,FALL,2024,ARCH,2330,1,2024-12-16 08:00:00,2024-12-16 11:00:00,TBA,TBA
4,FALL,2024,ARCH,2330,3,2024-12-16 08:00:00,2024-12-16 11:00:00,TBA,TBA
5,FALL,2024,ARCH,2350,1,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
6,FALL,2024,ARCH,2350,2,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
7,FALL,2024,ARCH,2350,3,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
8,FALL,2024,ARCH,2350,4,2024-12-17 08:00:00,2024-12-17 11:00:00,TBA,TBA
9,FALL,2024,ARCH,2510,1,2024-12-16 08:00:00,2024-12-16 11:00:00,TBA,TBA
